In [1]:
#This part of code, the goal is to go from mapping and county level weather data to to subgroup data
# Define paths
data_and_output_dir = '/Users/ansonkong/Downloads/Data for nyu work/'
full_future_data_path = '/Users/ansonkong/Downloads/rcp85hotter/'
full_historical_data_path = '/Users/ansonkong/Downloads/historic/'

#this just smooth out operation
cutoff_year=2020 #The year prediction dataset start
start_year=2000 #The year historical 
end_year=2100

#Debug mode(only output once)
debug=False

The goal of this version is no need for mapping

In [2]:
#import require file
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
import os
from datetime import datetime
from glob import glob

ba_dir = data_and_output_dir+'Input/EIA 930/BA/'


# Function to read and merge CSVs from a directory
def read_and_merge_csvs(directory, cols):
    combined_df = pd.DataFrame()
    for file in os.listdir(directory):
        if file.endswith('.csv'):
            df = pd.read_csv(os.path.join(directory, file), usecols=cols)
            combined_df = pd.concat([combined_df, df], ignore_index=True)
    return combined_df

ba_cols = ['UTC Time at End of Hour', 'Balancing Authority', 'Demand (MW)']
ba_combined_df = read_and_merge_csvs(ba_dir, ba_cols)



#Path, this will update itself
mapping_path = data_and_output_dir + 'output/merged_rb_control_area_mapping.csv'
future_population_data_path = data_and_output_dir + 'input/Electric_Retail_Service_Territories/ssp3_county_population.csv'
historical_population_data_path = data_and_output_dir + 'input/county_populations_2000_to_2020.csv'
output_path = data_and_output_dir + 'output/'

# Load mapping and population data
mapping_df = pd.read_csv(mapping_path)
future_population_df = pd.read_csv(future_population_data_path)
historical_population_df = pd.read_csv(historical_population_data_path)

#crate looping for interp
years = np.arange(cutoff_year, end_year + 1, 10)
interp_years = np.arange(cutoff_year, end_year + 1)

# Ensure FIPS codes are five zero-padded strings
historical_population_df['FIPS'] = historical_population_df['county_FIPS'].astype(str).str.zfill(5)
future_population_df['FIPS'] = future_population_df['FIPS'].astype(str).str.zfill(5)
mapping_df['GEOID'] = mapping_df['GEOID'].astype(str).str.zfill(5)

# Rename 'pop_{year}' columns to just '{year}'
for year in range(start_year, cutoff_year+1):
    if f'pop_{year}' in historical_population_df.columns:
        historical_population_df.rename(columns={f'pop_{year}': str(year)}, inplace=True)



# Assuming 'FIPS' is the column to join on and 'Year' is the column indicating the year in future_population_df
for index, row in future_population_df.iterrows():
    pop_values = [row[str(year)] for year in years if str(year) in row]
    interpolator = interp1d(years, pop_values, kind='linear', fill_value="extrapolate")
    interpolated_values = interpolator(interp_years)
    for year in interp_years:
        future_population_df.at[index, str(year)] = interpolated_values[year - cutoff_year]


# Select relevant columns for historical data
historical_population_df = historical_population_df[['FIPS'] + [str(year) for year in range(start_year, cutoff_year)]]

# Select relevant columns for future data
future_population_df = future_population_df[['FIPS'] + [str(year) for year in interp_years]]

# Concatenate historical and future dataframes
combined_population_df = pd.merge(historical_population_df, future_population_df, on='FIPS', how='outer')

/opt/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_3152/1708899298.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_3152/1708899298.py:17: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, file), usecols=cols)
/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0

In [3]:

serives_area_path = data_and_output_dir + '/Input/service_area'
# Read the CSV file
serives_area_df = read_and_merge_csvs(serives_area_path, ['BA_Code','County_FIPS','Year'])


In [4]:
import pandas as pd

# Assuming serives_area_df and combined_population_df are already defined

# Step 1: Add a placeholder for the population lookup
serives_area_df['Population'] = 0
serives_area_df['County_FIPS']=serives_area_df['County_FIPS'].astype(str).str.zfill(5)
# Step 2: Populate 'Population' in serives_area_df by looking up in combined_population_df
for idx, row in serives_area_df.iterrows():
    county_fips = row['County_FIPS']
    year = row['Year']
    
    # Find the matching row in combined_population_df by 'FIPS'
    matching_population_row = combined_population_df[combined_population_df['FIPS'] == county_fips]
    if not matching_population_row.empty and str(year) in combined_population_df.columns:
        # If a matching row is found and the year column exists in combined_population_df,
        # retrieve the population value
        population_value = matching_population_row.iloc[0][str(year)]
        serives_area_df.at[idx, 'Population'] = population_value

# Step 3: Compute 'Total_Population' for each combination of 'Year' and 'BA_Code' in serives_area_df
serives_area_df['Total_Population'] = serives_area_df.groupby(['Year', 'BA_Code'])['Population'].transform('sum')

# The 'Total_Population' column now contains the summed population for each combination of Year and BA_Code
serives_area_df


/var/folders/8v/68yqll_54lzgb57b5q8_fbrc0000gn/T/ipykernel_3152/2799943440.py:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '74434.99509211467' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  serives_area_df.at[idx, 'Population'] = population_value


,Year,County_FIPS,BA_Code,Population,Total_Population
0,2020,23003,NBSO,74434.995092,3.757132e+05
1,2020,23009,NBSO,53103.777044,3.757132e+05
2,2020,23019,NBSO,158886.075450,3.757132e+05
3,2020,23021,NBSO,18456.013671,3.757132e+05
4,2020,23027,NBSO,38974.016788,3.757132e+05
...,...,...,...,...,...
26339,2015,48483,SWPP,5664.000000,1.879542e+07
26340,2015,48485,SWPP,131034.000000,1.879542e+07
26341,2015,48487,SWPP,13067.000000,1.879542e+07
26342,2015,48499,SWPP,43119.000000,1.879542e+07


In [5]:
# import pandas as pd

# # Assuming ba_combined_df, serives_area_df, and aggregated_df are already defined
# # ba_combined_df needs 'UTC Time at End of Hour', 'Balancing Authority', and 'Demand (MW)' columns
# # serives_area_df should already have the 'Total_Population' calculated for each BA_Code and Year combination

# # Step 1: Create a Year column in ba_combined_df
# ba_combined_df['Year'] = pd.to_datetime(ba_combined_df['UTC Time at End of Hour']).dt.year

# # Prepare the result list
# expanded_results = []
# # Initialize a set to store details of records where a match is not found or total_population is zero
# unmatched_records = set()

# # Step 2 & 3: Iterate through ba_combined_df and expand with matching records
# for index, row in ba_combined_df.iterrows():
#     year = row['Year']
#     ba_code = row['Balancing Authority']

#     try:
#         # Attempt to directly convert or replace commas and then convert
#         demand_mw = float(str(row['Demand (MW)']).replace(',', ''))
#     except ValueError:
#         print(f"Cannot convert Demand (MW) to float for BA {ba_code} at {row['UTC Time at End of Hour']}. Using 0 as placeholder.")
#         demand_mw = 0  # Use a placeholder value or handle this scenario as appropriate
    
    
#     # Find matching rows in serives_area_df
#     matching_rows = serives_area_df[(serives_area_df['BA_Code'] == ba_code) & (serives_area_df['Year'] == year)]

#     if matching_rows.empty:
#         # If no matching rows are found, add the details to the unmatched_records set
#         unmatched_records.add((year, ba_code, 'No FIPS - no matching row', demand_mw))
#         continue  # Skip to the next row in ba_combined_df
#     population = float(match_row['Population']) if pd.notna(match_row['Population']) else 0
#     total_population = float(match_row['Total_Population']) if pd.notna(match_row['Total_Population']) else 0
    
#     for _, match_row in matching_rows.iterrows():
#         population = match_row['Population']
#         total_population = match_row['Total_Population']
        
#         if total_population > 0:  # Avoid division by zero
#             adjusted_demand = (population / total_population) * demand_mw
#             expanded_results.append({
#                 'UTC Time at End of Hour': row['UTC Time at End of Hour'],
#                 'County_FIPS': match_row['County_FIPS'],
#                 'BA_Code': ba_code,
#                 'Demand': adjusted_demand
#             })
#         else:
#             # Handle the case of zero total population
#             unmatched_records.add((year, ba_code, match_row['County_FIPS'], demand_mw))
#             print(f'Error with {row["UTC Time at End of Hour"]} for county {match_row["County_FIPS"]} - zero total population.')

# # Optionally, after processing
# for unmatched in unmatched_records:
#     print(f"Unmatched record: Year {unmatched[0]}, BA_Code {unmatched[1]}, County_FIPS {unmatched[2]}, Demand_MW {unmatched[3]}")

# # Step 4: Convert expanded_results to DataFrame
# expanded_results_df = pd.DataFrame(expanded_results)

# # Optional: You might want to group by 'UTC Time at End of Hour', 'County_FIPS', 'BA_Code' and sum 'Demand' if necessary
# # This step depends on whether you want to aggregate demands at this level
# final_results_df = expanded_results_df.groupby(['UTC Time at End of Hour', 'County_FIPS', 'BA_Code'], as_index=False)['Demand'].sum()

# # Save to CSV
# final_results_df.to_csv(data_and_output_dir + 'output/final_expanded_results.csv', index=False)


In [30]:
print('go')
# Define paths
output_csv_path = data_and_output_dir+'output/final_expanded_results_demand.csv'  # Update this path
checkpoint_path = data_and_output_dir+'output/checkpoint_for_demand.txt'  # Update this path

# Function to get the last processed index from the checkpoint file
def get_last_processed_index(checkpoint_path):
    try:
        with open(checkpoint_path, 'r') as f:
            last_index = int(f.read())
    except FileNotFoundError:
        last_index = -1
    return last_index

# Function to update the checkpoint file with the last processed index
def update_checkpoint(index, checkpoint_path):
    with open(checkpoint_path, 'w') as f:
        f.write(str(index))

# Get the last processed index
last_processed_index = get_last_processed_index(checkpoint_path)
print(last_processed_index)

# Check if the output CSV already exists; if not, write headers first
# Check if the output CSV already exists; if not, write headers first
if last_processed_index == -1:
    if not os.path.exists(output_csv_path):  # Check if file exists
        # Assuming the DataFrame structure, adjust column names as necessary
        pd.DataFrame(columns=['UTC Time at End of Hour', 'County_FIPS', 'BA_Code', 'Demand']).to_csv(output_csv_path, index=False)

# Step 1: Create a Year column in ba_combined_df
ba_combined_df['Year'] = pd.to_datetime(ba_combined_df['UTC Time at End of Hour']).dt.year

# Prepare the result list
expanded_results = []
# Initialize a set to store details of records where a match is not found or total_population is zero
unmatched_records = set()

# Process rows starting from the last checkpoint
for index, row in ba_combined_df.iterrows():
    if index <= last_processed_index:
        continue  # Skip already processed rows
    
    year = row['Year']
    ba_code = row['Balancing Authority']

    try:
        # Attempt to directly convert or replace commas and then convert
        demand_mw = float(str(row['Demand (MW)']).replace(',', ''))
    except ValueError:
        print(f"Cannot convert Demand (MW) to float for BA {ba_code} at {row['UTC Time at End of Hour']}. Using 0 as placeholder.")
        demand_mw = 0  # Use a placeholder value or handle this scenario as appropriate
    
    
    # Find matching rows in serives_area_df
    matching_rows = serives_area_df[(serives_area_df['BA_Code'] == ba_code) & (serives_area_df['Year'] == year)]

    if matching_rows.empty:
        # If no matching rows are found, add the details to the unmatched_records set
        unmatched_records.add((year, ba_code, 'No FIPS - no matching row'))
        continue  # Skip to the next row in ba_combined_df

    
    
    
    for _, match_row in matching_rows.iterrows():
        population = float(match_row['Population']) if pd.notna(match_row['Population']) else 0
        total_population = float(match_row['Total_Population']) if pd.notna(match_row['Total_Population']) else 0
        
        if total_population > 0:  # Avoid division by zero
            adjusted_demand = (population / total_population) * demand_mw
            expanded_results.append({
                'UTC Time at End of Hour': row['UTC Time at End of Hour'],
                'County_FIPS': match_row['County_FIPS'],
                'BA_Code': ba_code,
                'Demand': adjusted_demand
            })
                # Append the result to the CSV file
            with open(output_csv_path, 'a', newline='') as f:
                pd.DataFrame([{
                    'UTC Time at End of Hour': row['UTC Time at End of Hour'],
                    'County_FIPS': match_row['County_FIPS'],
                    'BA_Code': ba_code,
                    'Demand': adjusted_demand
                }]).to_csv(f, header=False, index=False)
        else:
            # Handle the case of zero total population
            unmatched_records.add((year, ba_code, match_row['County_FIPS']))

    # Update checkpoint after each successfully processed row
    update_checkpoint(index, checkpoint_path)
# Optionally, after processing
for unmatched in unmatched_records:
        print(f"Unmatched record: Year {unmatched[0]}, BA_Code {unmatched[1]}, County_FIPS {unmatched[2]}")



go
675


KeyboardInterrupt: 

In [19]:
expanded_results

[{'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01001',
  'BA_Code': 'SOCO',
  'Demand': 98.30539622135481},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01003',
  'BA_Code': 'SOCO',
  'Demand': 314.88901211436473},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01005',
  'BA_Code': 'SOCO',
  'Demand': 49.73608603677222},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01007',
  'BA_Code': 'SOCO',
  'Demand': 41.38085528923942},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01009',
  'BA_Code': 'SOCO',
  'Demand': 102.88150104427437},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01011',
  'BA_Code': 'SOCO',
  'Demand': 20.58007815787452},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'County_FIPS': '01013',
  'BA_Code': 'SOCO',
  'Demand': 38.503927429533405},
 {'UTC Time at End of Hour': '09/19/2020 12:00:00 AM',
  'C

In [9]:
matching_rows

,Year,County_FIPS,BA_Code,Population,Total_Population
6353,2019,01001,SOCO,55869.0,16834163.0
6354,2019,01003,SOCO,223234.0,16834163.0
6355,2019,01005,SOCO,24686.0,16834163.0
6356,2019,01007,SOCO,22394.0,16834163.0
6357,2019,01009,SOCO,57826.0,16834163.0
...,...,...,...,...,...
6599,2019,28111,SOCO,11973.0,16834163.0
6600,2019,28123,SOCO,28124.0,16834163.0
6601,2019,28129,SOCO,15916.0,16834163.0
6602,2019,28131,SOCO,18336.0,16834163.0


In [29]:
# Load the dataset
result_file_path = output_csv_path
results_df = pd.read_csv(result_file_path)

# Filter for entries with the specified UTC Time and BA Code
filtered_results = results_df[
    (results_df['UTC Time at End of Hour'] == '01/01/2018 7:00:00 AM') &
    (results_df['BA_Code'] == 'AEC')
]

# Sum the 'Demand' for the filtered entries
demand_sum = filtered_results['Demand'].sum()
demand_sum

748.0

In [17]:
results_df['UTC Time at End of Hour'][0]

'01/01/2018 7:00:00 AM'